## **Interactive Plasma Concentration Visualization**


### **1. Introduction**

##### 1.1 Fundamentals of Pharmacokinetics

Pharmacokinetics is the study of how drugs move through the body, including absorption, distribution, metabolism, and excretion. Plasma concentration (the amount of drug in the bloodstream at a given time) serves as a key measure of a medication's presence and activity in the body. For optimal therapeutic outcomes, many medications require plasma concentrations within a specific range:

- **Below minimum effective concentration (MEC)**: Insufficient therapeutic effect
- **Within therapeutic window**: Optimal clinical effect
- **Above maximum tolerated concentration**: Increased risk of adverse effects

Different drug formulations (immediate release, sustained release, extended release) are designed to achieve specific plasma concentration profiles to optimize therapeutic benefit.

##### 1.2 Modeling using Bateman Function

The Bateman function provides a mathematical model for describing drug concentration in plasma over time following oral administration. This first-order kinetic model balances two competing processes, absorption phase and elimination phase:

$$C(t) = A \cdot (e^{-K_e \cdot t} - e^{-K_a \cdot t})$$

Where:
- $C(t)$ = Plasma concentration at time $t$
- $A$ = Coefficient based on dose and distribution volume
- $K_e$ = Elimination rate constant (related to $T_{1/2}$)
- $K_a$ = Absorption rate constant
- $t$ = Time since dose administration

The model is characterized by three key pharmacokinetic parameters:

- $T_{max}$: Time to maximum concentration (hours)
- $C_{max}$: Peak plasma concentration (ng/ml)
- $T_{1/2}$: Elimination half-life (hours)

##### 1.3 Solving the Bateman Function

To apply the Bateman function in practice, we need to determine the function parameters from known pharmacokinetic values:

1. **Elimination rate constant** ($K_e$): Calculated from ${T_{1/2}}$, since drug elimination follows exponential decay model:
   $$C(t) = C_0 \cdot e^{-K_e \cdot t}$$

   At $t = T_{1/2}$, $C(t) = \frac{C_0}{2}$, leading to:
   $$\frac{C_0}{2} = C_0 \cdot e^{-K_e \cdot T_{1/2}}$$
   $$K_e = \frac{\ln(2)}{T_{1/2}}$$

2. **Absorption rate constant** ($K_a$): Calculated from $T_{max}$ and $K_e$, since at peak concentration ($t = T_{max}$), the derivative of the concentration function is zero:
   $$\frac{dC(t)}{dt}\bigg|_{t=T_{max}} = 0$$
   $$\frac{K_a e^{-K_a T_{max}} - K_e e^{-K_e T_{max}}}{e^{-K_e T_{max}} - e^{-K_a T_{max}}} = 0$$
   Solving for $K_a$:
   $$\frac{\ln(K_a) - \ln(K_e)}{K_a - K_e} = T_{max}$$

3. **Coefficient** ($A$): Once $K_a$ and $K_e$ are known, $A$ is determined using $C_{max}$ and $T_{max}$:
   $$A = \frac{C_{max}}{e^{-K_e T_{max}} - e^{-K_a T_{max}}}$$

##### 1.4 Extended Implementation with Metabolic Factor

This implementation extends the basic Bateman model by incorporating several advanced features:

1. **Metabolic Factor**: An adjustment parameter that accounts for individual variations in drug metabolism, affecting how quickly the drug is absorbed and eliminated
   - Adjusted $T_{max} = T_{max} / \text{Metabolic Factor}$
   - Adjusted $C_{max} = C_{max} / \sqrt{\text{Metabolic Factor}}$
   - Adjusted $K_e = K_e \times \text{Metabolic Factor}$

2. **Multiple-dose regimens**: Simulating realistic medication schedules with varying frequency

3. **Flexible dosing patterns**:
   - Variable dosing frequency and interval throughout the day
   - Skip-day dosing patterns
   - Customizable initial dose timing

4. **Advanced metrics**:
   - Area Under the Curve (AUC) calculations for total drug exposure
   - Average concentration measurements
   - End-of-day concentration tracking

This tool allows for exploration of how different dosing strategies affect plasma concentration profiles, supporting evidence-based decisions for medication administration schedules.

### **2. Code Blocks**

In [37]:
# %pip install numpy scipy matplotlib
%pip install -q ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [38]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import fsolve
from scipy.integrate import simpson

from ipywidgets import interactive, Layout

##### 2.1 Display Utilities

In [39]:
class AdditionalData:
    """
    Class to hold additional data for display.
    """

    class Data:
        pass

    data = Data

    @staticmethod
    def add_data(key, value):
        setattr(AdditionalData.data, key, value)

    @staticmethod
    def print_data():

        print("====================Additional Data=======================")
        print("==========================================================")
        print(
            f"Adjust for metabolic: Tmax={AdditionalData.data.t_max_adjusted:.2f},",
            f"Cmax={AdditionalData.data.c_max_adjusted:.2f},",
            f"T_half={AdditionalData.data.t_half_adjusted:.2f}",
        )
        print(
            f"Adjust for metabolic: A={AdditionalData.data.a_adjusted:.4f},",
            f"ke={AdditionalData.data.ke_adjusted:.4f},",
            f"ka={AdditionalData.data.ka_adjusted:.4f}",
        )

        print("==========================================================")
        print(
            "Overall Average Plasma Concentration (ng/ml):",
            f"{AdditionalData.data.avg_concen_all:.2f}",
        )
        print(
            "Plasma Volume AUC per Day (ng/ml):",
            f"{AdditionalData.data.avg_auc :.2f}",
        )
        print("==========================================================")


def float_to_time_str_5min(hours):
    """
    Convert hours (float) to time string with 5-minute precision.
    """

    # Convert to minutes and round to nearest 5
    total_minutes = int(round(hours * 60 / 5) * 5)

    # Extract hours and minutes components
    hour_part = total_minutes // 60
    minute_part = total_minutes % 60

    # Format as HH:MM
    return f"{hour_part:02d}:{minute_part:02d}"

##### 2.2 Bateman Function Calculations

In [40]:
def adjust_bateman_param(t_max, c_max, t_half, metabolic_factor):
    """
    Adjust Bateman function parameters based on metabolic rate.
    """
    # Calculate elimination rate constant and adjust for metabolic
    ke = np.log(2) / t_half
    ke_adjusted = ke * metabolic_factor

    # Adjust parameters based on metabolic rate
    t_max_adjusted = t_max / metabolic_factor
    c_max_adjusted = c_max / np.sqrt(metabolic_factor)
    t_half_adjusted = np.log(2) / ke_adjusted

    # Estimate absorption rate constant Ka from adjusted t_max and ke
    # From the condition that dC/dt = 0 at t = t_max
    def equation(ka_var):
        return (np.log(ka_var) - np.log(ke_adjusted)) / (
            ka_var - ke_adjusted
        ) - t_max_adjusted

    ka_adjusted = fsolve(equation, 0.1, maxfev=50000)[0]

    # Calculate coefficient A from the Bateman equation
    # C(t) = A * (e^(-Ke*t) - e^(-Ka*t))
    a_adjusted = c_max_adjusted / (
        np.exp(-ke_adjusted * t_max_adjusted) - np.exp(-ka_adjusted * t_max_adjusted)
    )

    # Store adjusted parameters for later
    AdditionalData.add_data("t_max_adjusted", t_max_adjusted)
    AdditionalData.add_data("c_max_adjusted", c_max_adjusted)
    AdditionalData.add_data("t_half_adjusted", t_half_adjusted)
    AdditionalData.add_data("a_adjusted", a_adjusted)
    AdditionalData.add_data("ke_adjusted", ke_adjusted)
    AdditionalData.add_data("ka_adjusted", ka_adjusted)

    return (a_adjusted, ke_adjusted, ka_adjusted, t_half_adjusted)


def calculate_bateman_data(
    t_half,
    a_coefficient,
    ke,
    ka,
    initial_dose_time,
    dose_interval,
    doses_per_day,
    skip_after_period,
    total_days,
):
    """
    Calculate plasma concentration data using the Bateman function for multiple doses.
    """
    # Calculate total simulation points
    total_time = 2 * t_half + 24 * total_days  # Total simulation time in hours
    data_points = doses_per_day * total_days * 1000  # Resolution for smooth curve

    # Create time array
    time_points = np.linspace(0, total_time, data_points)

    # Define inner function for Bateman equation
    def bateman(t, dose_time=0):
        time_since_dose = t - dose_time
        time_since_dose = np.maximum(
            time_since_dose, 0
        )  # concentration is zero before dose
        return a_coefficient * (
            np.exp(-ke * time_since_dose) - np.exp(-ka * time_since_dose)
        )

    # Initialize concentration array
    concentration = np.zeros_like(time_points)

    # Add contribution of each dose to the concentration
    for day in range(total_days):
        # Check if dosing should be skipped for this day
        if day > 0 and skip_after_period > 0:
            if day % (skip_after_period + 1) == skip_after_period:
                continue

        for dose_num in range(doses_per_day):
            # Calculate time for this specific dose
            current_dose_time = initial_dose_time + dose_num * dose_interval
            if current_dose_time > 24:
                raise ValueError(
                    f"Dose time {float_to_time_str_5min(current_dose_time)} exceeds 24 hours"
                )

            absolute_dose_time = 24 * day + current_dose_time

            # Add the effect of this dose
            concentration += bateman(time_points, absolute_dose_time)

    # Calculate indices for the end of each day
    points_per_day = int(data_points * 24.0 / total_time)
    midnight_indices = [points_per_day * (i + 1) for i in range(total_days)]

    # Calculate average concentration
    day_points_count = points_per_day * total_days
    avg_concen_all = np.mean(concentration[0:day_points_count])

    # Calculate average concentration at bedtime
    # Extract concentrations at the end of each day
    concen_midnight = [concentration[idx] for idx in midnight_indices]
    avg_concen_end = np.mean(concen_midnight)

    # Calculate average AUC using Simpson's rule
    auc = simpson(concentration[0:day_points_count], x=time_points[0:day_points_count])
    avg_auc = auc / total_days

    # Calculate actual_t_max
    max_concen_all = np.max(concentration)
    actual_t_max = time_points[np.argmax(concentration)]

    # Store adjusted parameters for later
    AdditionalData.add_data("avg_auc", avg_auc)
    AdditionalData.add_data("avg_concen_all", avg_concen_all)

    return (
        time_points,
        concentration,
        total_time,
        max_concen_all,
        avg_concen_end,
        actual_t_max,
    )

##### 2.3 Plotting Utilities

In [41]:
def create_title_info(
    total_days,
    doses_per_day,
    initial_dose_time,
    dose_interval,
    skip_after_period,
    t_max,
    c_max,
    t_half,
    metabolic_factor,
):
    """Create descriptive title for the plot."""
    # Basic parameters info
    title_info = (
        f"T_max={t_max:.1f} h, C_max={c_max:.1f} ng/ml, "
        f"T_half={t_half:.1f} h, Metabolic Factor={metabolic_factor:.2f}\n"
    )

    # Dosing schedule info
    title_info += (
        f"{total_days}-Day Administration, {doses_per_day}-Time Daily Dosing "
        f"Starts {float_to_time_str_5min(initial_dose_time)}"
    )

    # Add interval info if multiple doses per day
    if doses_per_day > 1:
        title_info += f" with {dose_interval:.1f}-Hour Interval"

    # Add skip period info if applicable
    if skip_after_period > 0:
        title_info += f", Skip for 1 Day After {skip_after_period}-Day Period"

    return title_info


def setup_x_axis_ticks(ax, total_time):
    """Configure x-axis ticks to show days."""
    tick_positions = np.arange(0, total_time, 8)
    tick_labels = [
        str(i // 3) if i % 3 == 0 else "" for i in range(len(tick_positions))
    ]

    ax.set_xticks(tick_positions)
    ax.set_xticklabels(tick_labels)
    ax.set_xlabel("Days")
    ax.margins(x=0)


def enhance_x_grid_lines(ax):
    """Enhance grid lines to better visualize day boundaries."""
    xgridlines = ax.get_xgridlines()
    for i, gridline in enumerate(xgridlines):
        if i % 3 == 0:
            gridline.set_linewidth(1.5)
        else:
            gridline.set_linewidth(0.5)


def add_max_concen_marker(ax, max_concen, total_days):
    """Add a horizontal line and label for maximum concentration."""
    x_min, x_max = ax.get_xlim()
    x_text = x_min + (x_max - x_min) * 0.005

    max_label = "Adjusted C_max" if total_days == 1 else f"Max within {total_days}d"
    ax.axhline(
        y=max_concen, color="red", linestyle="--", linewidth=1.5, label=max_label
    )
    ax.text(
        x=x_text,
        y=max_concen,
        s=f"{max_concen:.2f}",
        color="red",
        va="center",
        ha="left",
        fontsize=10,
        bbox=dict(facecolor="white", alpha=0.9, edgecolor="none", pad=0),
    )


def add_avg_concen_marker(ax, avg_concen, total_days):
    """Add a horizontal line and label for average concentration."""
    x_min, x_max = ax.get_xlim()
    x_text = x_min + (x_max - x_min) * 0.005

    ax.axhline(
        y=avg_concen,
        color="g",
        linestyle="--",
        linewidth=1.5,
        label=f"Midnight Average within {total_days}d",
    )
    ax.text(
        x=x_text,
        y=avg_concen,
        s=f"{avg_concen:.2f}",
        color="g",
        va="center",
        ha="left",
        fontsize=10,
        bbox=dict(facecolor="white", alpha=0.9, edgecolor="none", pad=0),
    )


def add_t_max_marker(ax, actual_t_max):
    """Add a vertical line and label for T_max."""
    y_min, y_max = ax.get_ylim()
    y_text = y_min + (y_max - y_min) * 0.01

    ax.axvline(
        x=actual_t_max, color="y", linestyle="--", linewidth=1.5, label="Adjusted T_max"
    )
    ax.text(
        x=actual_t_max,
        y=y_text,
        s=float_to_time_str_5min(actual_t_max),
        color="y",
        va="center",
        ha="left",
        fontsize=10,
        bbox=dict(facecolor="white", alpha=0.9, edgecolor="none", pad=0),
    )

##### 2.4 Pharmacokinetics Data Plotting

In [42]:
def create_plot(
    time_points,
    concentration,
    total_time,
    total_days,
    doses_per_day,
    doses_interval,
    skip_after_period,
    initial_dose_time,
    actual_t_max,
    max_concen_all,
    avg_concen_end,
    t_max,
    c_max,
    t_half,
    metabolic_factor,
    save_plot,
    y_limit,
):
    """Create and display a plot of plasma concentration over time."""
    fig, ax = plt.subplots(figsize=(18, 8))

    # Create title with dosing information
    title_info = create_title_info(
        total_days,
        doses_per_day,
        initial_dose_time,
        doses_interval,
        skip_after_period,
        t_max,
        c_max,
        t_half,
        metabolic_factor,
    )

    # Plot concentration curve
    ax.plot(time_points, concentration, label="Plasma Concentration Level")
    ax.set_xlabel("Time (hours)")
    ax.set_ylabel("Plasma Concentration (ng/mL)")
    ax.set_title("Plasma Concentration-Time Profile (Bateman Function)\n" + title_info)

    # Configure x-axis ticks
    setup_x_axis_ticks(ax, total_time)

    # Add markers for key values
    if total_days == 1:
        add_t_max_marker(ax, actual_t_max)

    add_max_concen_marker(ax, max_concen_all, total_days)
    add_avg_concen_marker(ax, avg_concen_end, total_days)

    # Enhance grid lines for better day visibility
    enhance_x_grid_lines(ax)

    # Set y-axis limit if specified
    if y_limit >= 100:
        ax.set_ylim(top=y_limit)

    # Finalize plot
    ax.legend()
    ax.grid(True)

    # Save figure if enabled
    if save_plot:
        plt.savefig("plasma_concentration_plot.png", dpi=300, bbox_inches="tight")

    plt.show()


def plot_plasma_concentration(
    save_plot,
    y_limit,
    t_max,
    c_max,
    t_half,
    metabolic_factor,
    total_days,
    doses_per_day,
    doses_interval,
    initial_dose_time,
    skip_after_period,
):
    """
    Calculate and visualize plasma concentration over time using the Bateman function.
    """
    # Calculate adjusted parameters based on metabolic_factor
    adjusted_params = adjust_bateman_param(
        t_max,
        c_max,
        t_half,
        metabolic_factor,
    )
    a_adjusted, ke_adjusted, ka_adjusted, t_half_adjusted = adjusted_params

    # Generate concentration data
    calculated_data = calculate_bateman_data(
        t_half_adjusted,
        a_adjusted,
        ke_adjusted,
        ka_adjusted,
        initial_dose_time,
        doses_interval,
        doses_per_day,
        skip_after_period,
        total_days,
    )
    (
        time_points,
        concentration,
        total_time,
        max_concen,
        avg_concen_end,
        actual_t_max,
    ) = calculated_data

    # Create and display plot
    create_plot(
        time_points,
        concentration,
        total_time,
        total_days,
        doses_per_day,
        doses_interval,
        skip_after_period,
        initial_dose_time,
        actual_t_max,
        max_concen,
        avg_concen_end,
        t_max,
        c_max,
        t_half,
        metabolic_factor,
        save_plot,
        y_limit,
    )

### **3. Interactive Visualization**

In [43]:
# Interactive widget descriptions
def get_widget_description(index):
    return [
        "Save Plot",
        "Y-axis Limit",
        "T_max (h)",
        "C_max (ng/ml)",
        "T_half (h)",
        "Metabolic Factor",
        "Total Days of Administration",
        "Doses per Day",
        "Doses Interval (h)",
        "Daily Initial Dose Time (h)",
        "Skip Dose After Period (d)",
    ][index]


# Wrapper for interactive function
def plot_wrapper(
    save_plot=False,
    y_limit=0,
    t_max=2,
    c_max=80,
    t_half=21,
    metabolic_factor=0.75,
    total_days=14,
    doses_per_day=1,
    doses_interval=6,
    initial_dose_time=8,
    skip_after_period=0,
):
    plot_plasma_concentration(**locals())
    AdditionalData.print_data()


# Create interactive widget
interactive_widget = interactive(
    plot_wrapper,
    save_plot=False,
    y_limit=(0, 500, 50),
    t_max=(0.5, 10.0, 0.5),
    c_max=(10.0, 200.0, 5.0),
    t_half=(1.0, 30.0, 0.5),
    metabolic_factor=(0.50, 2.0, 0.05),
    total_days=(1, 56, 1),
    doses_per_day=(1, 4, 1),
    doses_interval=(0.5, 16, 0.5),
    initial_dose_time=(6, 20, 1),
    skip_after_period=(0, 14, 1),
)

# Set layout for the interactive widget
for idx, widget in enumerate(
    interactive_widget.children[:-1]
):  # all children except the last one (output)
    widget.layout = Layout(width="50%")  # or use a percentage like '40%'
    widget.style.description_width = "200px"  # make the label wider
    widget.description = get_widget_description(idx)


# Display the interactive widget
interactive_widget

interactive(children=(Checkbox(value=False, description='Save Plot', layout=Layout(width='50%'), style=Checkbo…

### **4. Collected Data**

##### Sample B with Metabolic Factor = 0.75

| Type | Dose (mg) | Cmax (ng/ml) | Tmax (h) | T_half (h) |   Dosing   | Max (14d) | Midnight Avg (14d) | Average (14d) | AUC/d (14d) |
|:----:|:---------:|:------------:|:--------:|:----------:|:----------:|:---------:|:------------------:|:-------------:|:-----------:|
|  IR  |    37.5   |      40      |    2     |     21     |  2/d, q.8h |    192    |        152         |      146      |     3504    |
|  IR  |    50     |      55      |    2     |     21     |  2/d, q.8h |    264    |        209         |      201      |     4818    |
|  IR  |    75     |      80      |    2     |     21     |     1/d    |    209    |        137         |      148      |     3552    |
|  IR  |    75     |      80      |    2     |     21     |  2/d, q.8h |    384    |        305         |      292      |     7008    |
|  SR  |    100    |      85      |    3     |     21     |     1/d    |    224    |        152         |      162      |     3895    |
|  SR  |    150    |     130      |    3     |     21     |     1/d    |    342    |        233         |      248      |     5957    |
|  XL  |    150    |     120      |    5     |     21     |     1/d    |    324    |        236         |      244      |     5854    |
|  XL  |    150    |     120      |    5     |     21     |    1/2d    |    203    |        121         |      126      |     3035    |
|  XL  |    150    |     120      |    5     |     21     |    3/4d    |    295    |        184         |      190      |     4551    |
